In [1]:
# import libraries
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer

from paths import RAW_DIR, STAGE_DIR

### Import the data

In [2]:
tnm1 = pd.read_csv(RAW_DIR / "breast_cancer_data_tnm.csv")
tnm2 = pd.read_csv(RAW_DIR / "breast_cancer_data_tnm_2.csv")

In [3]:
# concatenate the two datasets
data_tnm = pd.concat([tnm1, tnm2])
data_tnm

,ehr,n_tumor,t,n,m,t_after_neoadj,n_after_neoadj,m_after_neoadj
0,6849,1,1,0.0,0.0,NaN,NaN,NaN
1,6849,3,2,0.0,0.0,NaN,NaN,NaN
2,268,1,1,0.0,0.0,NaN,NaN,NaN
3,268,2,1,1.0,0.0,NaN,NaN,NaN
4,268,3,2,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
24,6647,1,NaN,NaN,NaN,2.0,2.0,0.0
25,768,1,X,X,1.0,NaN,NaN,NaN
26,4534,1,2,0,0.0,NaN,NaN,NaN
27,7062,1,1,0,0.0,NaN,NaN,NaN


In [4]:
cat = (data_tnm.dtypes == object)
cat_cols = data_tnm.columns[cat].tolist()

for column in cat_cols:
  print(column, data_tnm[column].unique())

t ['1' '2' '0' 'IS' nan '3' '4' 'X']
n [0.0 1.0 2.0 3.0 nan '1' '2' '0' '3' 'X']
t_after_neoadj [nan '1' '0' '3' '2' 'IS' 1.0 2.0 0.0]


### NULL values

In [5]:
print(data_tnm.isnull().sum())
data_tnm.info()

ehr                 0
n_tumor             0
t                   4
n                   6
m                   4
t_after_neoadj    204
n_after_neoadj    204
m_after_neoadj    204
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 236 entries, 0 to 28
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ehr             236 non-null    int64  
 1   n_tumor         236 non-null    int64  
 2   t               232 non-null    object 
 3   n               230 non-null    object 
 4   m               232 non-null    float64
 5   t_after_neoadj  32 non-null     object 
 6   n_after_neoadj  32 non-null     float64
 7   m_after_neoadj  32 non-null     float64
dtypes: float64(3), int64(2), object(3)
memory usage: 16.6+ KB


In [6]:
data_temp = data_tnm.copy()

In [7]:
data_tnm['t'] = 'T' + (data_tnm['t']
    .fillna(9)
    .astype(str)
    .replace('\.0', '', regex=True))
data_tnm['n'] = 'N' + (data_tnm['n']
    .fillna(9)
    .astype(str)
    .replace('\.0', '', regex=True))
data_tnm['m'] = 'M' + (data_tnm['m']
    .fillna(9)
    .astype(str)
    .replace('\.0', '', regex=True))

data_tnm['t_after_neoadj'] = 'T' + (data_tnm['t_after_neoadj']
    .fillna(9)
    .astype(str)
    .replace('\.0', '', regex=True))
data_tnm['n_after_neoadj'] = 'N' + (data_tnm['n_after_neoadj']
    .fillna(9)
    .astype(str)
    .replace('\.0', '', regex=True))
data_tnm['m_after_neoadj'] = 'M' + (data_tnm['m_after_neoadj']
    .fillna(9)
    .astype(str)
    .replace('\.0', '', regex=True))

In [ ]:
# data_tnm['t'] = data_tnm['t'].fillna('X')
# data_tnm['n'] = pd.to_numeric(data_tnm['n'], errors='coerce', downcast='integer').fillna('X')
# data_tnm['m'] = pd.to_numeric(data_tnm['m'], errors='coerce', downcast='integer').fillna('X')
# # data_tnm['t_after_neoadj'] = pd.to_numeric(data_tnm['t_after_neoadj'], errors='ignore', downcast='integer')
# data_tnm['n_after_neoadj'] = pd.to_numeric(data_tnm['n_after_neoadj'], errors='coerce', downcast='integer').fillna('X')
# # data_tnm['m_after_neoadj'] = pd.to_numeric(data_tnm['m_after_neoadj'], errors='coerce', downcast='integer').fillna('X')

In [ ]:
# data_tnm['t_after_neoadj'].fillna('X', inplace=True)
# data_tnm['m_after_neoadj'] = data_tnm['m_after_neoadj'].astype('object').fillna('X')
# data_tnm['n_after_neoadj'] = data_tnm['n_after_neoadj'].astype('object').fillna('X')
data_tnm
# to_numeric()

In [ ]:
print(data_tnm.info())
data_tnm.isnull().sum()

In [ ]:
# Users can be duplicated for diff tumor types
data_tnm[data_tnm.ehr.duplicated(keep=False)]

In [ ]:
cat = (data_tnm.dtypes == object)
cat_cols = data_tnm.columns[cat].tolist()

for column in cat_cols:
  print(column, data_tnm[column].unique())

In [8]:
data_tnm.loc[data_tnm.t.eq("TIS"), ['t', 't_after_neoadj']].value_counts(dropna=False)

t    t_after_neoadj
TIS  T9                16
dtype: int64

In [ ]:
len(data_tnm['ehr'].unique())

In [ ]:
data_tnm['t'].value_counts()

In [ ]:
data_tnm['t_after_neoadj'].value_counts()

In [ ]:
# data_tnm['t_old'] = data_temp['t'].copy()
# data_tnm['n_old'] = data_temp['n'].copy().astype(str)
# data_tnm['m_old'] = data_temp['m'].copy()
# data_tnm['t_after_neoadj_old'] = data_temp['t_after_neoadj'].copy().astype(str)
# data_tnm['n_after_neoadj_old'] = data_temp['n_after_neoadj'].copy().astype(str)
# data_tnm['m_after_neoadj_old'] = data_temp['m_after_neoadj'].copy()

In [9]:
data_tnm.to_parquet(STAGE_DIR / 'tnm-preprocessed-v1.parquet')